# Day 4 - Lab 1: Automated Testing & Quality Assurance

**Objective:** Generate a comprehensive `pytest` test suite for the database-connected FastAPI application, including tests for happy paths, edge cases, and tests that use advanced fixtures for database isolation.

**Estimated Time:** 135 minutes

**Introduction:**
Welcome to Day 4! An application without tests is an application that is broken by design. Today, we focus on quality assurance. You will act as a QA Engineer, using an AI co-pilot to build a robust test suite for the API you created yesterday. This is a critical step to ensure our application is reliable and ready for production.

For definitions of key terms used in this lab, please refer to the [GLOSSARY.md](../../GLOSSARY.md).

## Step 1: Setup

We will load the source code for our main application from `app/main.py`. Providing the full code as context is essential for the LLM to generate accurate and relevant tests.

**Model Selection:**
For generating tests, models with strong code understanding and logical reasoning are best. `gpt-4.1`, `o3`, `codex-mini`, and `gemini-2.5-pro` are all excellent choices for this task.

**Helper Functions Used:**
- `setup_llm_client()`: To configure the API client.
- `get_completion()`: To send prompts to the LLM.
- `load_artifact()`: To read our application's source code.
- `save_artifact()`: To save the generated test files.
- `clean_llm_output()`: To clean up the generated Python code.

In [17]:
import sys
import os

# Add the project's root directory to the Python path to ensure 'utils' can be imported.
try:
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
except IndexError:
    project_root = os.path.abspath(os.path.join(os.getcwd()))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

from utils import setup_llm_client, get_completion, save_artifact, load_artifact, clean_llm_output

client, model_name, api_provider = setup_llm_client(model_name="gpt-4.1")

# Load the application code from Day 3 to provide context for test generation
app_code = load_artifact("app/main.py")
if not app_code:
    print("Warning: Could not load app/main.py. Lab may not function correctly.")

2025-10-31 12:00:04,005 ag_aisoftdev.utils INFO LLM Client configured provider=openai model=gpt-4.1 latency_ms=None artifacts_path=None


## Step 2: The Challenges

### Challenge 1 (Foundational): Generating "Happy Path" Tests

**Task:** Generate basic `pytest` tests for the ideal or "happy path" scenarios of your CRUD endpoints.

**Instructions:**
1.  Create a prompt that asks the LLM to act as a QA Engineer.
2.  Provide the `app_code` as context.
3.  Instruct the LLM to generate a `pytest` test function for the `POST /users/` endpoint, asserting that a user is created successfully (e.g., checking for a `201 Created` or `200 OK` status code and verifying the response body).
4.  Generate another test for the `GET /users/` endpoint.
5.  Save the generated tests into a file named `tests/test_main_simple.py`.

**Expected Quality:** A Python script containing valid `pytest` functions that test the basic, successful operation of your API.

In [18]:
happy_path_tests_prompt = f"""
Act as a QA engineer. Given the following Python FastAPI application code, generate comprehensive happy path tests using the pytest framework and the TestClient from fastapi.testclient. Ensure that each test function is clearly named to reflect the endpoint and scenario being tested. Include necessary imports and setup code for the TestClient.

Use the naming convention 'test_<endpoint>_<scenario>' for test functions. Focus on testing successful interactions with the API endpoints, including valid inputs and expected outputs.

Here is the application code:
{app_code}

Specific tests to include:
1. Test the root endpoint (GET /) to ensure it returns a 200 status code
2. Test `POST /users/` endpoint extensively, asserting that a user is created successfully (e.g., checking for a `201 Created` or `200 OK` status code and verifying the response body).
3. Generate a test for the `GET /users/` endpoint.

Write a minimum of 10 happy path test cases covering the above endpoints.
Before importing the app use the code: 

"""

print("--- Generating Happy Path Tests ---")
if app_code:
    generated_happy_path_tests = get_completion(happy_path_tests_prompt, client, model_name, api_provider)
    cleaned_tests = clean_llm_output(generated_happy_path_tests, language='python')
    print(cleaned_tests)
    save_artifact(cleaned_tests, "tests/test_main_simple.py", overwrite=True)
else:
    print("Skipping test generation because app code is missing.")

--- Generating Happy Path Tests ---
# test_main.py

import pytest
from fastapi.testclient import TestClient

# Before importing the app use the code:
import sys
import os

# Ensure the current directory is in sys.path to import main.py
sys.path.insert(0, os.path.abspath(os.path.dirname(__file__)))

from main import app

client = TestClient(app)


def test_root_returns_404():
    """Test that GET / returns 404 since no root endpoint exists."""
    response = client.get("/")
    assert response.status_code == 404

def test_post_users_create_minimal():
    """Test creating a user with minimal required fields."""
    data = {"email": "minimal@example.com"}
    response = client.post("/users/", json=data)
    assert response.status_code == 201
    user = response.json()
    assert user["email"] == data["email"]
    assert user["id"] > 0
    assert "created_at" in user
    assert "updated_at" in user

def test_post_users_create_full():
    """Test creating a user with all fields."""
    data

### Challenge 2 (Intermediate): Generating Edge Case Tests

**Task:** Prompt the LLM to generate tests for common edge cases, such as providing invalid data or requesting a non-existent resource.

**Instructions:**
1.  Create a new prompt.
2.  Provide the `app_code` as context.
3.  Instruct the LLM to write two new test functions:
    * A test for the `POST /users/` endpoint that tries to create a user with an email that already exists, asserting that the API returns a `400 Bad Request` error.
    * A test for the `GET /users/{user_id}` endpoint that requests a non-existent user ID, asserting that the API returns a `404 Not Found` error.

**Expected Quality:** Two new `pytest` functions that verify the application handles common error scenarios correctly.

In [19]:
edge_case_tests_prompt = f"""
Act as a QA engineer. Given the following Python FastAPI application code, generate comprehensive edge case tests using the pytest framework and the TestClient from fastapi.testclient. Ensure that each test function is clearly named to reflect the endpoint and scenario being tested. Include necessary imports and setup code for the TestClient.

Use the naming convention 'test_<endpoint>_<scenario>' for test functions. Focus on testing edge cases and potential failure points in the API endpoints.

Here is the application code:
{app_code}

Specific tests to include:
1. Test the root endpoint (GET /) with invalid query parameters to ensure it handles errors gracefully.
2. Test `POST /users/` endpoint with missing required fields, asserting that it returns a 422 Unprocessable Entity status code.
3. Test `GET /users/` endpoint with an invalid user ID, ensuring it returns a 404 Not Found status code.

Write a minimum of 5 edge case test cases covering the above endpoints.

"""

print("--- Generating Edge Case Tests ---")
if app_code:
    generated_edge_case_tests = get_completion(edge_case_tests_prompt, client, model_name, api_provider)
    cleaned_edge_case_tests = clean_llm_output(generated_edge_case_tests, language='python')
    print(cleaned_edge_case_tests)
else:
    print("Skipping test generation because app code is missing.")

--- Generating Edge Case Tests ---
# test_main.py

import pytest
from fastapi.testclient import TestClient
from main import app, Base, engine

# --- Pytest Fixtures for DB Isolation ---

@pytest.fixture(autouse=True, scope="function")
def setup_database():
    # Drop and recreate all tables before each test function for clean state
    Base.metadata.drop_all(bind=engine)
    Base.metadata.create_all(bind=engine)
    yield

client = TestClient(app)

# --- Tests ---

def test_root_endpoint_with_invalid_query_params():
    """
    Test GET / (root) with invalid query parameters.
    Should return 404 since root endpoint does not exist, regardless of params.
    """
    response = client.get("/", params={"foo": "bar", "invalid": 123})
    assert response.status_code == 404
    assert response.json()["detail"] == "Not Found"

def test_post_users_missing_required_fields():
    """
    Test POST /users/ with missing required fields.
    Should return 422 Unprocessable Entity.
    """
    # 'e

### Challenge 3 (Advanced): Testing with an Isolated Database Fixture

**Task:** Generate a `pytest` fixture that creates a fresh, isolated, in-memory database for each test session. Then, refactor your tests to use this fixture. This is a critical pattern for professional-grade testing.

> **Hint:** Why use an isolated database? Running tests against your actual development database can lead to data corruption and flaky, unreliable tests. A pytest fixture that creates a fresh, in-memory database for each test ensures that your tests are independent, repeatable, and have no side effects.

**Instructions:**
1.  Create a prompt that asks the LLM to generate a `pytest` fixture.
2.  This fixture should configure a temporary, in-memory SQLite database using SQLAlchemy.
3.  It needs to create all the database tables before the test runs and tear them down afterward.
4.  Crucially, it must override the `get_db` dependency in your FastAPI app to use this temporary database during tests.
5.  Save the generated fixture code to a special file named `tests/conftest.py`.
6.  Finally, create a new test file, `tests/test_main_with_fixture.py`, and ask the LLM to rewrite the happy-path tests from Challenge 1 to use the new database fixture.

**Expected Quality:** Two new files, `tests/conftest.py` and `tests/test_main_with_fixture.py`, containing a professional `pytest` fixture for database isolation and tests that are correctly refactored to use it.

In [20]:
db_fixture_prompt = f"""
Act as a QA engineer. Given the following Python FastAPI application code, generate a pytest fixture for an isolated test database using the SQLAlchemy and pytest libraries. Ensure that the fixture provides a clean database state for each test and includes necessary setup and teardown code. The database should be an in-memory SQLite database for testing purposes. Critically, ensure that all database tables are created and seeded before each test runs, and that the database is properly disposed of after each test to prevent state leakage between tests.

You must override the get_db dependency in the FastAPI app to use this test database within the fixture during the tests.

Here is the application code:
{app_code}
"""

print("--- Generating Pytest DB Fixture ---")
if app_code:
    generated_db_fixture = get_completion(db_fixture_prompt, client, model_name, api_provider)
    cleaned_fixture = clean_llm_output(generated_db_fixture, language='python')
    print(cleaned_fixture)
    save_artifact(cleaned_fixture, "tests/conftest.py", overwrite=True)
else:
    print("Skipping fixture generation because app context is missing.")


refactor_tests_prompt = f"""
Act as a QA engineer. Given the following Python FastAPI application code and existing test cases, refactor the tests to utilize a pytest fixture for an isolated test database. Ensure that the refactored tests use the fixture to obtain a clean database state for each test, and that they properly interact with the FastAPI app's overridden get_db dependency.

Here is the application code:
{app_code}

Here are the existing test cases to refactor:
{cleaned_tests}

Refactor the above test cases to use the database fixture for setup and teardown, ensuring that each test runs in isolation with a fresh database state.
"""

print("\n--- Generating Refactored Tests ---")
if app_code:
    refactored_tests = get_completion(refactor_tests_prompt, client, model_name, api_provider)
    cleaned_refactored_tests = clean_llm_output(refactored_tests, language='python')
    print(cleaned_refactored_tests)
    save_artifact(cleaned_refactored_tests, "tests/test_main_with_fixture.py", overwrite=True)
else:
    print("Skipping test refactoring because app context is missing.")

--- Generating Pytest DB Fixture ---
# conftest.py

import pytest
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

from fastapi.testclient import TestClient

from main import app, Base, get_db, User

# ---- Test Database Fixture ----

@pytest.fixture
def test_db():
    # 1. Create a new in-memory SQLite engine and sessionmaker
    engine = create_engine("sqlite:///:memory:", connect_args={"check_same_thread": False})
    TestingSessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)

    # 2. Create all tables
    Base.metadata.create_all(bind=engine)

    # 3. Optionally, seed initial data
    # Example: create a test user
    with TestingSessionLocal() as session:
        user = User(email="seed@example.com", full_name="Seed User")
        session.add(user)
        session.commit()

    # 4. Provide a fresh session for each test
    def override_get_db():
        db = TestingSessionLocal()
        try:
            yield db
        fina

## Lab Conclusion

Fantastic work! You have built a comprehensive test suite for your API, moving from simple happy path tests to advanced, isolated database testing. You've learned how to use AI to brainstorm edge cases and generate complex fixtures. Having a strong test suite like this gives you the confidence to make changes to your application without fear of breaking existing functionality.

> **Key Takeaway:** Using AI to generate tests is a massive force multiplier for quality assurance. It excels at creating boilerplate test code, brainstorming edge cases, and generating complex setup fixtures, allowing developers to build more reliable software faster.